In [14]:
from elasticsearch import Elasticsearch, helpers
import csv

es = Elasticsearch("http://localhost:9200")
print(es.ping())

True


In [15]:
# with open('NewData.csv',encoding="utf8") as f:
#     reader = csv.DictReader(f)
#     helpers.bulk(es, reader, index='news')
    
import pandas as pd
import json

df = pd.read_csv("NewData.csv",encoding="utf8")
json_str = df.to_json(orient='records')

json_records = json.loads(json_str)

es = Elasticsearch()
index_name = 'news'
doctype = 'census_record'
es.indices.delete(index=index_name, ignore=[400, 404])
es.indices.create(index=index_name, ignore=400)
action_list = []
for row in json_records:
    record ={
        '_op_type': 'index',
        '_index': index_name,
        '_type' : doctype,
        '_source': row
    }
    action_list.append(record)
helpers.bulk(es, action_list)


C:\Users\Zahra\AppData\Roaming\Python\Python311\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
C:\Users\Zahra\AppData\Roaming\Python\Python311\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


(2631, [])

In [16]:
word = input("Please Enter a word :")
res = es.search(index="news", body={"query": {"match_phrase": {"Body" : word}}})['hits']['hits']

for item in res:
    strWord = item['_source']['Body']
    print(strWord.find(word))


Please Enter a word :متخلفان
118
261
730


In [19]:
import persian
def es_iterate_all_documents(es, index, pagesize=250, **kwargs):
    offset = 0
    while True:
        result = es.search(index=index, **kwargs, body={
            "size": pagesize,
            "from": offset
        })
        hits = result["hits"]["hits"]
        # Stop after no more docs
        if not hits:
            break
        # Yield each entry
        yield from (hit['_source'] for hit in hits)
        # Continue from there
        offset += pagesize
        
for entry in es_iterate_all_documents(es, 'news'):
    numPersian = persian.convert_fa_numbers(entry['Pdate'])
    if(numPersian.find('12') != -1):
        print(entry['News Code'],'-',entry['Pdate'])
    elif(numPersian.find('13') != -1):
        print(entry['News Code'],'-',entry['Pdate'])
    

کد خبر ۸۳۵۶۶۱ - تاریخ انتشار: ۲۲:۱۳ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۶۳۱ - تاریخ انتشار: ۱۸:۱۲ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۶۳۲ - تاریخ انتشار: ۱۸:۱۲ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۲۳ - تاریخ انتشار: ۱۵:۱۲ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۵۹ - تاریخ انتشار: ۱۴:۱۳ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۵۱ - تاریخ انتشار: ۱۳:۵۶ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۲۸ - تاریخ انتشار: ۱۳:۵۰ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۱۹ - تاریخ انتشار: ۱۳:۵۰ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۴۹ - تاریخ انتشار: ۱۳:۴۲ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۴۶ - تاریخ انتشار: ۱۳:۳۸ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۴۸ - تاریخ انتشار: ۱۳:۳۸ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۳۷ - تاریخ انتشار: ۱۳:۳۱ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۲۱ - تاریخ انتشار: ۱۳:۳۰ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۴۷ - تاریخ انتشار: ۱۳:۲۹ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۴۵ - تاریخ انتشار: ۱۳:۲۳ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۴۴ - تاریخ انتشار: ۱۳:۲۰ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۱۴ - تاریخ انتشار: ۱۳:۲۰ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۴۳ - تاریخ انتشار: ۱۳:۱۹ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۴۰ - تاریخ انتشار: ۱۳:۱۶ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۳۰ - تاریخ انتشار: ۱۳:۱۵ - ۲۸-۰۱-۱۴۰۱
کد خبر ۸۳۵۵۴۱ - تاری

کد خبر ۸۳۳۸۹۸ - تاریخ انتشار: ۱۰:۱۳ - ۱۸-۰۱-۱۴۰۱
کد خبر ۸۳۳۸۵۰ - تاریخ انتشار: ۱۹:۱۳ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۸۸ - تاریخ انتشار: ۱۳:۵۸ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۸۶ - تاریخ انتشار: ۱۳:۵۴ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۸۵ - تاریخ انتشار: ۱۳:۵۲ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۸۴ - تاریخ انتشار: ۱۳:۴۱ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۸۳ - تاریخ انتشار: ۱۳:۳۸ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۸۲ - تاریخ انتشار: ۱۳:۳۶ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۸۱ - تاریخ انتشار: ۱۳:۲۹ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۸۰ - تاریخ انتشار: ۱۳:۱۰ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۶۸ - تاریخ انتشار: ۱۳:۰۶ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۷۹ - تاریخ انتشار: ۱۳:۰۵ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۷۸ - تاریخ انتشار: ۱۳:۰۰ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۷۷ - تاریخ انتشار: ۱۲:۵۵ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۷۶ - تاریخ انتشار: ۱۲:۴۹ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۷۵ - تاریخ انتشار: ۱۲:۴۸ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۷۴ - تاریخ انتشار: ۱۲:۴۶ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۶۵ - تاریخ انتشار: ۱۲:۴۶ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۷۲ - تاریخ انتشار: ۱۲:۴۴ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۷۳ - تاریخ انتشار: ۱۲:۳۹ - ۱۷-۰۱-۱۴۰۱
کد خبر ۸۳۳۷۷۱ - تاری